## Content based recommendation system

Content based recommender systems do not make use of data from other users to recommend a movie. Instead, they utilize a descriptive set of attributes such as keywords or the summary of a movie. The disadvantage is that these systems will recommend the same movie to the user, based on the input. However, they could be useful in recommending a movie that not many people have seen or rated.  

In the content based recommendation system it is only the user that plays a role in the recommendation. This method can also be combined with collaborative filtering methods.

In this notebook, the text content of the movies from the `merged` dataset is going to be alalyzed. The goal is to rank all the movies in the dataset based on a similarity measure with the input movie. For similarity measures, the cosine similarity will be used. Moreover, the content comes from the movies plots and possibly also the keywords. In order to remove the most common words, TF-IDF is used. Finally, the input to the TF-IDF algorithm will be the lemmatized text from each movie's content.

In [1]:
# Common libraries imports
import pandas as pd

In [2]:
# Not as common libraries imports and installation. 
# !python3 -m pip install nltk ## For linux and not environment
# !pip install nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Since it is a good idea to remove stop words from tf idf calculations, as stated also in [Chapter 1.3.1 MMDS](http://mmds.org/), a list of English stop words is created:

In [35]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

Read the `merged` dataset, or its 'cleaned' version, that has duplicates removed

In [3]:
# a = data.sort_values('Release Year', ascending=False).drop_duplicates(subset=['Title', 'Release Year'], keep='last')
# a.loc[a.Title=='The Mask']['release_date']

In [4]:
data = pd.read_csv('../Data/data_cleaned.csv')

In [5]:
def print_info(index):
    '''
    Helper function used for an initial overview of the dataset
    '''
    print(f"Title:\n{data.iloc[index]['Title']}\n")
    print(f"Release Year:\n{data.iloc[index]['Release Year']}\n")
    print(f"Link:\n{data.iloc[index]['Wiki Page']}\n")
    print(f"Tagline:\n{data.iloc[index]['tagline']}\n")
    print(f"Overview:\n{data.iloc[index]['overview']}\n")
    print(f"Summary:\n{data.iloc[index]['Plot']}\n")

As an example, use `print_info` for a random movie:

In [6]:
print_info(12456)

Title:
Me and the Colonel

Release Year:
1958

Link:
https://en.wikipedia.org/wiki/Me_and_the_Colonel

Tagline:
nan

Overview:
Jacobowsky, a Jewish refugee, flees from the Nazis with an aristocratic, anti-semitic Polish officer trying to get papers to England. Jurgens learns to appreciate Jacobowsky, despite their competition for the same woman, and together they outwit their pursuers

Summary:
In Paris during the World War II invasion of France by Nazi Germany, Jewish refugee S. L. Jacobowsky (Danny Kaye) seeks to leave the country before it falls. Meanwhile, Polish diplomat Dr. Szicki (Ludwig Stössel) gives antisemitic, autocratic Polish Colonel Prokoszny (Curt Jürgens) secret information that must be delivered to London by a certain date.
The resourceful Jacobowsky, who has had to flee from the Nazis several times previously, manages to "buy" an automobile from the absent Baron Rothschild's chauffeur. Prokoszny peremptorily requisitions the car, but finds he must accept an unwelcome

Which text should we use as content? We can use `tagline` as an alternative title, `overview` which is a sentence that summarizes the movie and `Plot`, the summary of the movie. The latter is in general a longer text. We can use either the latter or for each movie create a txt document that contains the desired text.  

In the following, as a prototype, I am only using the `Plot`.

There are 2 ways to normalize text:Stemming and Lemmatization. The difference can be found [here](https://www.guru99.com/stemming-lemmatization-python-nltk.html). In the following I am using Lemmatization.  

The procedure is as follows. Lemmatize each movie's text content, get the frequency for each movie's lemmas and then use TF-IDF. To this end, I create a dataframe to store, the movie title, the relase year and the lemmas as list of words for each movie.

In [39]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer

In [8]:
tokenizer = RegexpTokenizer(r'\w+') # Remove punctuation
wordnet_lemmatizer = WordNetLemmatizer() # Create lemmatizer

# text = "studies studying cries cry"
# tokenization = nltk.word_tokenize(text)
# for w in tokenization:
#     print("Lemma for {} is {}".format(w, wordnet_lemmatizer.lemmatize(w)))  

In [40]:
def create_lemmas_list(content_txt):
    lemmas = []    
    tokenization = tokenizer.tokenize(content_txt.lower()) # Lowercase the whole text, to avoid dealing with case
    for w in tokenization:
        # Do not consider single characters. Can be resolved via tf-idf,
        # but maybe there are single characters due to wrong line breaks.
        if w in stop_words:
            continue
        lemmas.append(wordnet_lemmatizer.lemmatize(w, wordnet.VERB))
    
    return lemmas

Test this:

In [41]:
txt = data.iloc[459].Plot
a = create_lemmas_list(txt)

In [47]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'adult', 'belongs_to_collection',
       'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'Release Year', 'Title',
       'Origin/Ethnicity', 'Director', 'Cast', 'Genre', 'Wiki Page', 'Plot'],
      dtype='object')

In [48]:
# Create a df to hold the movies and the tokenized text
movie_plots_tokens_df = data[['Title', "Plot"]]

In [51]:
def tokenize(text):
    tokenization = tokenizer.tokenize(text.lower())
    tokens = [ wordnet_lemmatizer.lemmatize(token, wordnet.VERB) for token in tokenization if token not in stop_words ]
    return tokens

In [55]:
movie_plots_tokens_df['Tokens'] = movie_plots_tokens_df['Plot'].apply(tokenize)

C:\Users\user\AppData\Local\Temp2\ipykernel_4840\863147497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_plots_tokens_df['Tokens'] = movie_plots_tokens_df['Plot'].apply(tokenize)


In [56]:
movie_plots_tokens_df.Tokens

0        [2006, four, friends, jess, scarlett, johansso...
1        [bhairava, kala, bhairava, telugu, version, vi...
2        [present, day, paris, diana, receive, photogra...
3        [du, qiu, chinese, lawyer, defeat, many, legal...
4        [1950s, feral, puppy, name, toby, whisk, away,...
                               ...                        
16224    [rarebit, fiend, gorge, welsh, rarebit, restau...
16225    [scenes, introduce, use, line, poem, 2, santa,...
16226    [film, open, two, bandits, break, railroad, te...
16227    [alice, follow, large, white, rabbit, rabbit, ...
16228    [earliest, know, adaptation, classic, fairytal...
Name: Tokens, Length: 16229, dtype: object

In [33]:
def tf(document):
    doc_tokens = create_lemmas_list(document)
    freq_dist = nltk.FreqDist(doc_tokens)
    
    # FreqDist return the dictionary sorted in descending order.
    # I did not find this explicitly in the docs, so I find the max freq in the document
    max_freq = sorted( list(freq_dist.values()), reverse=True )[0]
    
    tfs = {token:freq/max_freq for (token, freq) in freq_dist.items() }
    return tfs

In [59]:
from tqdm.notebook import tqdm

def df(data_frame):
    df_dict = {}
    for i in tqdm(range(len(data_frame))):
        for token in set(data_frame.iloc[i].Tokens):
            for j in range(len(data_frame)):
                document = set(data_frame.iloc[j].Tokens)
                if token in document:
                    df_dict[token] = df_dict.get(token,[j]) + [j]
    return df_dict

In [60]:
df(movie_plots_tokens_df)

  0%|          | 0/16229 [00:00<?, ?it/s]

KeyboardInterrupt: 